In [1]:
import tensorflow as tf
import numpy as np
import os
from skimage.io import imread, imshow
from skimage.transform import resize
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Conv2DTranspose
# tqdm - for showing progress
from tqdm import tqdm

In [2]:
TRAIN_PATH = "NuclieDataset/stage1_train"
TEST_PATH = "NuclieDataset/stage1_test"

In [4]:
TRAIN_IMAGES_FOLDERS = os.listdir(TRAIN_PATH)
n = len(TRAIN_IMAGES_FOLDERS)

IMG_WIDTH, IMG_HEIGHT, COLOR_CHANNEL = 128, 128, 3

X_train = np.zeros((n, IMG_HEIGHT, IMG_WIDTH, COLOR_CHANNEL), dtype=np.uint8)
Y_train = np.zeros((n, IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)

C:\Users\asus\AppData\Local\Temp\ipykernel_32920\3600249695.py:7: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y_train = np.zeros((n, IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)


In [5]:
n

670

In [6]:
X_train.shape

(670, 128, 128, 3)

In [7]:
img_path = TRAIN_PATH + "/" + TRAIN_IMAGES_FOLDERS[0] + "/images/"

In [8]:
img_path

'NuclieDataset/stage1_train/00071198d059ba7f5914a526d124d28e6d010c92466da21d4a04cd5413362552/images/'

In [ ]:
for i in tqdm(range(len(TRAIN_IMAGES_FOLDERS))):
    img_path = TRAIN_PATH + "/" + TRAIN_IMAGES_FOLDERS[i] + "/images/"
    img_name = os.listdir(img_path)[0]
    img_final_path = img_path + "/" + img_name
    img = imread(img_final_path)[:,:,:COLOR_CHANNEL]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode="constant",
                preserve_range=True)
    X_train[i] = img
    mask_path = TRAIN_PATH + "/" + TRAIN_IMAGES_FOLDERS[i] + "/masks/" 
    mask_images = os.listdir(mask_path)
    mask = np.zeros([IMG_HEIGHT, IMG_WIDTH, 1], dtype=np.bool)
    for j in range(len(mask_images)):
        mask_img = imread(mask_path + "/" + mask_images[j])
        mask_img = resize(mask_img, (IMG_HEIGHT, IMG_WIDTH), mode="constant",
                         preserve_range=True)
        mask_img = np.expand_dims(mask_img, axis=-1)
        mask = np.maximum(mask, mask_img)
    
    Y_train[i] = mask

  0%|                                                                                          | 0/670 [00:00<?, ?it/s]C:\Users\asus\AppData\Local\Temp\ipykernel_32920\1738662445.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mask = np.zeros([IMG_HEIGHT, IMG_WIDTH, 1], dtype=np.bool)
 33%|██████████████████████████▌                                                     | 222/670 [02:17<02:06,  3.54it/s]